# Basic work

### Setup

In [1]:
var moment = require('moment');
var { constants } = require('../utils');
var { advanceTimeAndBlock } = require('../utils/evm');
var { toUnit } = require('../utils/bn');
var { LIQUIDITIES } = require('../utils/constants');
var { getLatestBlockTimestamp, getBlockTimestamp } = require('../utils/evm');
var { next, clear } = require('../utils/jupyter');
var { Notebook } = require('../utils/notebook');

clear();

In [2]:
var $ = new Notebook();

In [3]:
next(async () => {
    await $.setup();
});

In [16]:
next(async () => {
    oracleFactory = await $.fetch('Keep3rV2OracleFactory','0xaed599aadfee8e32cedb59db2b1120d33a7bacfd')
    oracle = await $.fetch('Keep3rV2Oracle','0xe20B3f175F9f4e1EDDf333f96b72Bba138c9e83a')
    uniV2Pair = await $.fetch('UniswapV2Pair','0xaf988aff99d3d0cb870812c325c588d8d8cb7de8')
});

In [51]:
oracle.length()

4720

In [43]:
oracle.observations(4719)

[
  1628025092,
  BigNumber { _hex: '0x03ddb747213929cbc5d7bc', _isBigNumber: true },
  BigNumber { _hex: '0xc43ee9c5303f22cf95820f', _isBigNumber: true },
  timestamp: 1628025092,
  price0Cumulative: BigNumber { _hex: '0x03ddb747213929cbc5d7bc', _isBigNumber: true },
  price1Cumulative: BigNumber { _hex: '0xc43ee9c5303f22cf95820f', _isBigNumber: true }
]

### updateable function
```js
const periodSize = 1800;
function updateable() external view returns (bool) {
    Observation memory _point = observations[length-1];
    (,, uint timestamp) = IUniswapV2Pair(pair).getReserves();
    uint timeElapsed = timestamp - _point.timestamp;
    return timeElapsed > periodSize;
}
```

In [40]:
next(async()=>{
    await oracle.connect(await $.impersonate(oracleFactory.address)).update({gasPrice:0})    
})

In [49]:
uniV2Pair.getReserves()

[
  BigNumber { _hex: '0x0539206908b596e8e78a', _isBigNumber: true },
  BigNumber { _hex: '0x2e026c45103e10d397', _isBigNumber: true },
  1628028923,
  _reserve0: BigNumber { _hex: '0x0539206908b596e8e78a', _isBigNumber: true },
  _reserve1: BigNumber { _hex: '0x2e026c45103e10d397', _isBigNumber: true },
  _blockTimestampLast: 1628028923
]

In [46]:
$.sleep(3600)

In [50]:
1628028923 - 1628025092 

3831

In [32]:
uniV2Pair.sync()

{
  hash: '0x9cf480ab467a21bdbc925016afb15535515e669ea793efdfef5295bfbe76a500',
  type: 0,
  accessList: null,
  blockHash: '0x05215876006e4c312784f0734c0484fb2fbe38fa5a940487edadf7ad91054b78',
  blockNumber: 12954373,
  transactionIndex: 0,
  confirmations: 1,
  from: '0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266',
  gasPrice: BigNumber { _hex: '0x01dcd65000', _isBigNumber: true },
  gasLimit: BigNumber { _hex: '0xaf09a8', _isBigNumber: true },
  to: '0xaf988afF99d3d0cb870812C325C588D8D8CB7De8',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  nonce: 44,
  data: '0xfff6cae9',
  r: '0x00683930a9eebe6f2e4e7427bddd3bde22b90f1a34cee6689fd6aad9db60b002',
  s: '0x0d3c8558665f29d28205e60421a2503fe5fef18dafc6bcda8d957d4f9297a4e7',
  v: 62709,
  creates: null,
  chainId: 31337,
  wait: [Function (anonymous)]
}

In [29]:
next(async()=>{
    // make a Swap
    await uniRouter.connect($.jobOwner).exactInputSingle([$.keep3rV1.address,WETH9,10000,$.jobOwner.address,2638281696,1,0,0])
})

In [28]:
$.sleep(100)

In [19]:
$.job.connect($.keeper).work();

Error: VM Exception while processing transaction: reverted with custom error 'InsufficientFunds()'

### Setup the KP3R/WETH Pool
`KP3R = 100`
`ETH = 5.2`
`20 KP3R = 1 ETH`
`Minted LPs = 22`

In [37]:
next(async () => {  
    
    // fund the jobOwner
    await weth.connect($.jobOwner).deposit({value: toUnit(100)})
    await $.keep3rV1Proxy.connect($.governance)['mint(address,uint256)']($.jobOwner.address,toUnit(0xffff))
  
    // approve spendings
    await weth.connect($.jobOwner).approve(kp3rPool.address,toUnit(0xffffff))
    await $.keep3rV1.connect($.jobOwner).approve(kp3rPool.address,toUnit(0xffffff))
    await $.keep3rV1.connect($.jobOwner).approve(uniRouter.address,toUnit(0xffffff))
    
    // mint a position
    await kp3rPool.connect($.jobOwner).mint(
        toUnit(100),
        toUnit(5.2),
        toUnit(90),
        toUnit(4),
        $.jobOwner.address)
    
    console.log('LPs', ((await kp3rPool.balanceOf($.jobOwner.address)).div(toUnit(1))).toNumber())
    
    // make an empty swap
    await uniRouter.connect($.jobOwner).exactInputSingle([$.keep3rV1.address,WETH9,10000,$.jobOwner.address,2638281696,1,0,0])
    await $.sleep($.time(5,'days'))
})

Error: VM Exception while processing transaction: reverted with reason string 'Price slippage check'
    at Keep3rV1Pair.burn (contracts/for-test/Keep3rV1Pair.sol:382)
    at Keep3rV1Pair.mint (contracts/for-test/Keep3rV1Pair.sol:349)
    at processTicksAndRejections (node:internal/process/task_queues:96:5)
    at runNextTicks (node:internal/process/task_queues:65:3)
    at listOnTimeout (node:internal/timers:526:9)
    at processTimers (node:internal/timers:500:7)
    at HardhatNode._mineBlockWithPendingTxs (/Users/levihernan/wonder/keep3r/notebooks/node_modules/hardhat/src/internal/hardhat-network/provider/node.ts:1261:23)
    at HardhatNode.mineBlock (/Users/levihernan/wonder/keep3r/notebooks/node_modules/hardhat/src/internal/hardhat-network/provider/node.ts:384:16)
    at EthModule._sendTransactionAndReturnHash (/Users/levihernan/wonder/keep3r/notebooks/node_modules/hardhat/src/internal/hardhat-network/provider/modules/eth.ts:1379:18)
    at HardhatNetworkProvider.request (/Users/l

### Add liquidity to job

In [33]:
next(async () => {
    await $.keep3r.connect($.governance).approveLiquidity(kp3rPool.address)

    await kp3rPool.approve($.keep3r.address, toUnit(0xffff))
    await $.keep3r.connect($.jobOwner).addLiquidityToJob($.job.address,kp3rPool.address,toUnit(2))
    console.log('liquidityAmount', ((await $.keep3r.liquidityAmount($.job.address, kp3rPool.address)).div(toUnit(1))).toNumber())
});

Error: VM Exception while processing transaction: reverted with custom error 'LiquidityPairApproved()'
    at Keep3r.approveLiquidity (contracts/peripherals/jobs/Keep3rJobFundableLiquidity.sol:132)
    at processTicksAndRejections (node:internal/process/task_queues:96:5)
    at runNextTicks (node:internal/process/task_queues:65:3)
    at listOnTimeout (node:internal/timers:526:9)
    at processTimers (node:internal/timers:500:7)
    at HardhatNode._mineBlockWithPendingTxs (/Users/levihernan/wonder/keep3r/notebooks/node_modules/hardhat/src/internal/hardhat-network/provider/node.ts:1261:23)
    at HardhatNode.mineBlock (/Users/levihernan/wonder/keep3r/notebooks/node_modules/hardhat/src/internal/hardhat-network/provider/node.ts:384:16)
    at EthModule._sendTransactionAndReturnHash (/Users/levihernan/wonder/keep3r/notebooks/node_modules/hardhat/src/internal/hardhat-network/provider/modules/eth.ts:1379:18)
    at HardhatNetworkProvider.request (/Users/levihernan/wonder/keep3r/notebooks/nod

### Work and burn

In [62]:
next(async () => {
    console.log('Start of simulation');
    
    // sleep some hours
    await $.sleepAndRecord($.time(2, 'weeks'), $.time(4, 'hours'));
    
    // work a lot
    for (let i = 0; i < 56; i++) {
        await $.recordCredits();
        await $.job.connect($.keeper).work();
        await $.sleepAndRecord($.time(0.5, 'days'),$.time(12,'hours'));
        bonds = await $.keep3r.bonds($.keeper.address,$.keep3rV1.address)
        if( bonds.gt(toUnit(20))) {
            console.log('swaps', (bonds.div(toUnit(1))).toNumber(), 'KP3R')
            await $.keep3r.connect($.keeper).unbond($.keep3rV1.address,bonds)            
            // somebody swaps 20 KP3R into the pool
            await uniRouter.connect($.jobOwner).exactInputSingle([
                $.keep3rV1.address,
                WETH9,
                10000,
                $.keeper.address,
                2638281696,
                bonds,
                toUnit(0.00001),
                0])
        }
    }
    
    await $.sleepAndRecord($.time(2, 'weeks'), $.time(4, 'hours'));
    
    console.log('End of simulation');
});

Start of simulation
swaps 20 KP3R
swaps 20 KP3R
swaps 20 KP3R
swaps 20 KP3R


In [64]:
next(async()=>{
    console.log(await uniPool.slot0())
})

[
  BigNumber { _hex: '0x0a4fc9b652cc301adac50f35', _isBigNumber: true },
  -64242,
  14,
  160,
  161,
  0,
  true,
  sqrtPriceX96: BigNumber { _hex: '0x0a4fc9b652cc301adac50f35', _isBigNumber: true },
  tick: -64242,
  observationIndex: 14,
  observationCardinality: 160,
  observationCardinalityNext: 161,
  feeProtocol: 0,
  unlocked: true
]


In [59]:
uniPool.observations(10)

[
  1661199956,
  BigNumber { _hex: '-0x018c28a57bce', _isBigNumber: true },
  BigNumber {
    _hex: '0x366c5f0000000001d9aa67f72c6869750724f5',
    _isBigNumber: true
  },
  true,
  blockTimestamp: 1661199956,
  tickCumulative: BigNumber { _hex: '-0x018c28a57bce', _isBigNumber: true },
  secondsPerLiquidityCumulativeX128: BigNumber {
    _hex: '0x366c5f0000000001d9aa67f72c6869750724f5',
    _isBigNumber: true
  },
  initialized: true
]

In [56]:
uniPool.observations(9)

[
  1646338722,
  BigNumber { _hex: '-0xcf0c0600e6', _isBigNumber: true },
  BigNumber {
    _hex: '0x366c5f00000000011dd9d6bc1589239b377683',
    _isBigNumber: true
  },
  true,
  blockTimestamp: 1646338722,
  tickCumulative: BigNumber { _hex: '-0xcf0c0600e6', _isBigNumber: true },
  secondsPerLiquidityCumulativeX128: BigNumber {
    _hex: '0x366c5f00000000011dd9d6bc1589239b377683',
    _isBigNumber: true
  },
  initialized: true
]

In [59]:
next(async () => {
    await $.draw();
});